# Imports
Import necessary libraries and modules for data handling, image processing, model building, and evaluation.

In [1]:
import os
import pandas as pd
from roboflow import Roboflow
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load Data
Load the dataset and labels. Also, initialize the Roboflow API for extracting PROI.

In [2]:
# Define the paths to the datasets
data_path = 'rsna-bone-age-dataset'
train_image_path = os.path.join(data_path, 'boneage-training-dataset')
test_image_path = os.path.join(data_path, 'boneage-test-dataset')

# Load the CSV files containing the labels
train_labels = pd.read_csv(os.path.join(data_path, 'boneage-training-dataset.csv'))
test_labels = pd.read_csv(os.path.join(data_path, 'boneage-test-dataset.csv'))

# Display the first few rows of the training and test datasets
print("Training Dataset:")
print(train_labels.head())
print("\nTest Dataset:")
print(test_labels.head())

# Initialize the Roboflow API
api_key = "It0WPkl2TkJgCXRF0PCh"
rf = Roboflow(api_key=api_key)
project = rf.workspace().project("x-ray-bones-detector")
model = project.version(3).model

Training Dataset:
     id  boneage   male
0  1377      180  False
1  1378       12  False
2  1379       94  False
3  1380      120   True
4  1381       82  False

Test Dataset:
   Case ID Sex
0     4360   M
1     4361   M
2     4362   M
3     4363   M
4     4364   M
loading Roboflow workspace...
loading Roboflow project...


# Image Processing
Define functions for image preprocessing, PROI extraction, bone segmentation.

In [3]:
# Function to preprocess an image
def preprocess_image(image):
    # Gamma Correction 
    gamma = 1.5
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    gamma_corrected_image = cv2.LUT(image, table)

    # Adaptive Histogram Equalization
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(image)

    # Image Fusion
    fused_image = cv2.addWeighted(image, 0.5, gamma_corrected_image, 0.25, 0)
    fused_image = cv2.addWeighted(fused_image, 0.75, clahe_image, 0.25, 0)

    # Noise Reduction
    denoised_image = cv2.fastNlMeansDenoising(fused_image, None, 10, 7, 21)

    return denoised_image

# Function to extract phalangeal regions of interest (PROI) from an image
def extract_proi(image):
    # Predict bones in image using Roboflow model
    result = model.predict(image, confidence=40).json()

    # Filter the predictions based on the class 
    proximal_proi = [pred for pred in result['predictions'] if pred['class'] == 'proximal phalanges']
    intermediate_proi = [pred for pred in result['predictions'] if pred['class'] == 'intermediate phalanges']
    distal_proi = [pred for pred in result['predictions'] if pred['class'] == 'distal phalanges']

    return proximal_proi, intermediate_proi, distal_proi

# Function to segment the bones from the phalangeal regions
def segment_bones(image, proi):
    # Segment the bones from the phalangeal regions
    segmeneted_bones = []
    for bone in proi:
        x = int(bone['x'] - bone['width'] / 2)
        y = int(bone['y'] - bone['height'] / 2)
        width = int(bone['width'])
        height = int(bone['height'])
        segmeneted_bone = image[y:y+height, x:x+width]

        # Normalize the segmented bone image to range [0, 1]
        segmeneted_bone = segmeneted_bone / 255.0

        segmeneted_bones.append(segmeneted_bone)

    return segmeneted_bones

# Training Data Preparation
Prepare the training dataset by iterating over the labels, preprocessing the images, extracting features, and splitting the dataset into training and validation sets.

In [4]:
# Prepare the training dataset
x_train = []
y_train = []

# Iterate over the training dataset
for index, row in train_labels.iterrows():
    img_path = os.path.join(train_image_path, str(row['id']) + '.png')
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    print(img_path)
    
    preprocessed_image = preprocess_image(image)

    proximal_proi, intermediate_proi, distal_proi = extract_proi(preprocessed_image)

    proximal_phalanges = segment_bones(preprocessed_image, proximal_proi)
    intermediate_phalanges = segment_bones(preprocessed_image, intermediate_proi)
    distal_phalanges = segment_bones(preprocessed_image, distal_proi)

    # Append the features to the training dataset
    x_train.append(proximal_phalanges + intermediate_phalanges + distal_phalanges)
    # Append the bone age to the target dataset
    y_train.append(row['boneage'])

# Split the training dataset into training and validation datasets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

rsna-bone-age-dataset\boneage-training-dataset\1377.png
rsna-bone-age-dataset\boneage-training-dataset\1378.png
rsna-bone-age-dataset\boneage-training-dataset\1379.png
rsna-bone-age-dataset\boneage-training-dataset\1380.png
rsna-bone-age-dataset\boneage-training-dataset\1381.png
rsna-bone-age-dataset\boneage-training-dataset\1382.png
rsna-bone-age-dataset\boneage-training-dataset\1383.png
rsna-bone-age-dataset\boneage-training-dataset\1384.png
rsna-bone-age-dataset\boneage-training-dataset\1385.png
rsna-bone-age-dataset\boneage-training-dataset\1387.png
rsna-bone-age-dataset\boneage-training-dataset\1388.png
rsna-bone-age-dataset\boneage-training-dataset\1389.png
rsna-bone-age-dataset\boneage-training-dataset\1390.png
rsna-bone-age-dataset\boneage-training-dataset\1391.png
rsna-bone-age-dataset\boneage-training-dataset\1393.png
rsna-bone-age-dataset\boneage-training-dataset\1394.png
rsna-bone-age-dataset\boneage-training-dataset\1395.png
rsna-bone-age-dataset\boneage-training-dataset\1

# CNN Model Definition
Defines the architecture of the CNN model using the VGG16 base model.

In [ ]:
# Load the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)

# Define the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='mean_absolute_error', metrics=['mean_absolute_error'])

# Model Training and Evaluation
Trains the CNN model on the training dataset and evaluates it on the validation dataset

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, checkpoint]
)

# Save the model
model.save('bone_age_prediction_model.h5')

# Load the best model
model.load_weights('best_model.h5')

# Evaluate the model
loss, mae = model.evaluate(x_val, y_val)
print("Validation Loss:", loss)
print("Validation MAE:", mae)

# Testing and Prediction
Prepares the test dataset and predicts bone age for the test images using the trained CNN model.

In [ ]:
# Prepare the test dataset
x_test = []

# Iterate over the test dataset
for index, row in test_labels.iterrows():
    img_path = os.path.join(test_image_path, str(row['Case ID']) + '.png')
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    preprocessed_image = preprocess_image(image)

    proximal_proi, intermediate_proi, distal_proi = extract_proi(preprocessed_image)

    proximal_phalanges = segment_bones(preprocessed_image, proximal_proi)
    intermediate_phalanges = segment_bones(preprocessed_image, intermediate_proi)
    distal_phalanges = segment_bones(preprocessed_image, distal_proi)

    # Append the features to the test dataset
    x_test.append(proximal_phalanges + intermediate_phalanges + distal_phalanges)

# Predict the bone age
y_test = model.predict(x_test)

# Save the predictions to a CSV file
test_labels['Predicted Bone Age'] = y_test
test_labels.to_csv('predictions.csv', index=False)

# Display the first few rows of the predictions
print("Predictions:")
print(test_labels.head())